# Monetary Policy - Currencies

## Imports

In [241]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import pmp_functions_v4 as pmp

import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

path = "../../Data_Ryan"

## Data Cleaning

In [242]:
# --- Load Riskfree Rate ---
factors_data = pd.read_excel(
    f"{path}/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100

riskfree = factors_data["RF"].resample('ME').last()
display(riskfree)

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_46804/711168680.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


1926-07-31    0.0022
1926-08-31    0.0025
1926-09-30    0.0023
1926-10-31    0.0032
1926-11-30    0.0031
               ...  
2025-06-30    0.0034
2025-07-31    0.0034
2025-08-31    0.0038
2025-09-30    0.0033
2025-10-31    0.0037
Freq: ME, Name: RF, Length: 1192, dtype: float64

In [243]:
# --- Load Factors Data ---
famafrench_data = pd.read_csv(
    f"{path}/famafrench_factors.csv",
    index_col = 0,
    parse_dates = True
)

famafrench_data.index = pd.to_datetime(famafrench_data.index, format='%Y%m')
famafrench_data.index = famafrench_data.index + pd.offsets.MonthEnd(0)
famafrench_data /= 100
display(famafrench_data)

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_46804/2041547973.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  famafrench_data = pd.read_csv(


,MKT-RF,SMB,HML,RMW,CMA,UMD,BAB
DATE,,,,,,,
1980-01-31,0.000550,0.000188,0.000185,-0.000184,0.000189,0.000745,0.000695
1980-02-29,-0.000123,-0.000162,0.000059,-0.000095,0.000292,0.000789,-0.000132
1980-03-31,-0.001289,-0.000697,-0.000096,0.000182,-0.000105,-0.000958,-0.001181
1980-04-30,0.000396,0.000105,0.000103,-0.000218,0.000034,-0.000048,0.000574
1980-05-31,0.000526,0.000200,0.000038,0.000043,-0.000063,-0.000118,0.000618
...,...,...,...,...,...,...,...
2025-06-30,0.000486,-0.000002,-0.000160,-0.000320,0.000145,-0.000264,0.000527
2025-07-31,0.000198,-0.000015,-0.000127,-0.000029,-0.000208,-0.000096,0.000184
2025-08-31,0.000185,0.000488,0.000442,-0.000068,0.000207,-0.000354,0.000646


In [244]:
# --- Benchmark Data ---
benchmark_data = pd.read_excel(
    f"{path}/Benchmarks.xlsx",
    index_col = 0,
    parse_dates = True
)
display(benchmark_data)
benchmark_data.index = pd.to_datetime(benchmark_data.index)
benchmark_data = benchmark_data.resample('ME').last()

benchmark_TR = benchmark_data[['MSCI World']].pct_change()
benchmark_XR = benchmark_TR.sub(riskfree, axis = 0)

benchmark_TR.columns = ['Benchmark Total Return']
benchmark_XR.columns = ['Benchmark Excess Return']
benchmark_returns = pd.concat([benchmark_TR, benchmark_XR], axis = 1).dropna()

display(benchmark_returns)

,S&P 500,MSCI World
Date,,
1986-12-31,242.1700,NaN
1987-01-30,274.7800,NaN
1987-02-27,285.6377,NaN
1987-03-31,293.8792,NaN
1987-04-30,291.2698,NaN
...,...,...
2025-07-31,14412.5527,8214.1572
2025-08-29,14704.7217,8431.0801
2025-09-30,15240.0381,8705.7139


,Benchmark Total Return,Benchmark Excess Return
1991-01-31,0.034364,0.029164
1991-02-28,0.090450,0.085650
1991-03-31,-0.031331,-0.035731
1991-04-30,0.005910,0.000610
1991-05-31,0.020740,0.016040
...,...,...
2025-06-30,0.043488,0.040088
2025-07-31,0.013121,0.009721
2025-08-31,0.026408,0.022608
2025-09-30,0.032574,0.029274


In [245]:
# --- Load Monetary Policy Data ---
monetary_data = pd.read_excel(
    f"{path}/2_year_yields.xlsx",
    index_col = 0,
    parse_dates = True
)
monetary_data.index = pd.to_datetime(monetary_data.index)
monetary_data.index = monetary_data.index + pd.offsets.MonthEnd(0)

display(monetary_data)

,US,UK,EU,JP,CH,AU,EM
Date,,,,,,,
1980-01-31,11.8790,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-29,14.7390,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-31,14.6390,NaN,NaN,NaN,NaN,NaN,NaN
1980-04-30,10.8190,NaN,NaN,NaN,NaN,NaN,NaN
1980-05-31,9.2590,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,3.7192,3.817,1.861,0.750,-0.048,3.2075,1.368
2025-07-31,3.9571,3.861,1.964,0.823,-0.082,3.3524,1.435
2025-08-31,3.6167,3.943,1.940,0.871,-0.103,3.3380,1.414


In [246]:
# --- Load Currency Prices ---
currency_data = pd.read_excel(
    f"{path}/FX Data.xlsx",
    sheet_name = 'RETURNS',
    index_col = 0,
    parse_dates = True
)
currency_data.index = pd.to_datetime(currency_data.index)
currency_data.index = currency_data.index + pd.offsets.MonthEnd(0)
currency_XR = currency_data

display(currency_XR)

,CH,EU,JP,UK,AU,EM
Date,,,,,,
1989-01-31,-0.062505,NaN,-0.044448,-0.028124,0.043550,NaN
1989-02-28,0.020885,NaN,0.025663,-0.002575,-0.095372,NaN
1989-03-31,-0.067754,NaN,-0.048472,-0.030821,0.027844,NaN
1989-04-30,-0.009532,NaN,-0.006180,0.004816,-0.025641,NaN
1989-05-31,-0.022300,NaN,-0.070709,-0.068691,-0.047106,NaN
...,...,...,...,...,...,...
2025-06-30,0.033185,0.036774,-0.003389,0.020141,0.022884,0.004021
2025-07-31,-0.027679,-0.033666,-0.048025,-0.038372,-0.024318,-0.009627
2025-08-31,0.010759,0.021636,0.021495,0.022178,0.017316,0.009796


# Global Variables

In [247]:
frequency = 1
t_cost = 0
# window = 12*20
short = True
beta_neutral = False
target_vol = 0.10
rf = riskfree
benchmark = benchmark_data

## Signal Generation

In [248]:
# --- Compute Monetary Policy Signal ---
monetary_policy_signal = monetary_data.diff(12)
display(monetary_policy_signal)

,US,UK,EU,JP,CH,AU,EM
Date,,,,,,,
1980-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,-1.0343,-0.404,-0.972,0.384,-0.852,-0.9559,-0.290
2025-07-31,-0.3004,0.035,-0.567,0.367,-0.742,-0.5227,-0.087
2025-08-31,-0.2998,-0.166,-0.451,0.497,-0.693,-0.3335,-0.120


In [249]:
# --- Currencies Composite Signal Construction ---
# Logic: For Currencies, we want increasing Monetary Policy Yields (+)
monetary_policy_signal = monetary_policy_signal

display(monetary_policy_signal)

,US,UK,EU,JP,CH,AU,EM
Date,,,,,,,
1980-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,-1.0343,-0.404,-0.972,0.384,-0.852,-0.9559,-0.290
2025-07-31,-0.3004,0.035,-0.567,0.367,-0.742,-0.5227,-0.087
2025-08-31,-0.2998,-0.166,-0.451,0.497,-0.693,-0.3335,-0.120


## Asset Returns

In [250]:
currency_XR['US'] = 0.0
display(currency_XR)

,CH,EU,JP,UK,AU,EM,US
Date,,,,,,,
1989-01-31,-0.062505,NaN,-0.044448,-0.028124,0.043550,NaN,0.0
1989-02-28,0.020885,NaN,0.025663,-0.002575,-0.095372,NaN,0.0
1989-03-31,-0.067754,NaN,-0.048472,-0.030821,0.027844,NaN,0.0
1989-04-30,-0.009532,NaN,-0.006180,0.004816,-0.025641,NaN,0.0
1989-05-31,-0.022300,NaN,-0.070709,-0.068691,-0.047106,NaN,0.0
...,...,...,...,...,...,...,...
2025-06-30,0.033185,0.036774,-0.003389,0.020141,0.022884,0.004021,0.0
2025-07-31,-0.027679,-0.033666,-0.048025,-0.038372,-0.024318,-0.009627,0.0
2025-08-31,0.010759,0.021636,0.021495,0.022178,0.017316,0.009796,0.0


## Portfolio Construction

In [251]:
# --- Ranking & Weighting ---
# Rank countries 1 to N for each month based on the raw signal.
# axis = 1 means we rank across columns (countries).

rank = monetary_policy_signal.rank(axis = 1, method = 'average', ascending = False)
display(rank)

,US,UK,EU,JP,CH,AU,EM
Date,,,,,,,
1980-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,7.0,3.0,6.0,1.0,4.0,5.0,2.0
2025-07-31,4.0,2.0,6.0,1.0,7.0,5.0,3.0
2025-08-31,4.0,3.0,6.0,1.0,7.0,5.0,2.0


In [252]:
# --- Standardize Ranks ---
# Convert ranks into Z-scores (Weights) that sum to zero.
# Weight = (Rank - Mean_Rank) / Std_Dev_Rank

rank_mean = rank.mean(axis = 1)
rank_std = rank.std(axis = 1)
standardized_weights = rank.sub(rank_mean, axis = 0).div(rank_std, axis = 0)

display(standardized_weights)

,US,UK,EU,JP,CH,AU,EM
Date,,,,,,,
1980-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,1.38873,-0.46291,0.92582,-1.38873,0.00000,0.46291,-0.92582
2025-07-31,0.00000,-0.92582,0.92582,-1.38873,1.38873,0.46291,-0.46291
2025-08-31,0.00000,-0.46291,0.92582,-1.38873,1.38873,0.46291,-0.92582


In [253]:
# --- Volatility Scaling (Risk Management) ---
# Step A: Calculate 'Raw' Strategy Returns (Before Vol Scaling)
# IMPORTANT: Shift weights by 1 to trade next month's return.
strategy_raw_ret = (standardized_weights.shift(1) * currency_XR).sum(axis=1)

# Step B: Forecast Volatility
# Calculate realized volatility over a 36-month rolling window (annualized)
# We use the raw strategy's realized vol to estimate future volatility.
expected_vol = strategy_raw_ret.rolling(window = 36).std() * np.sqrt(12)

# We use previous rolling volatility (shift 1) to size today's position
lev_factor = target_vol / expected_vol.shift(1)

## **⭐ CRITICAL CHANGE: Scaling the Weights**

# Step C: Estimate Portfolio Weights
# Apply the leverage factor to the standardized weights
# We use .mul(axis=0) to multiply the 2D DataFrame (weights) 
# by the 1D Series (lev_factor_series) along the rows (axis=0).
final_strategy_weights = standardized_weights.mul(lev_factor, axis=0).fillna(0).loc['1998-01-31':'2025-10-31']


display(final_strategy_weights)

,US,UK,EU,JP,CH,AU,EM
Date,,,,,,,
1998-01-31,0.140668,-0.140668,-0.703341,-0.422004,0.422004,0.703341,0.000000
1998-02-28,0.416479,-0.416479,-0.694132,-0.138826,0.138826,0.694132,0.000000
1998-03-31,0.426021,0.142007,-0.710035,-0.426021,-0.142007,0.710035,0.000000
1998-04-30,0.487946,-0.162649,-0.813243,0.162649,-0.487946,0.813243,0.000000
1998-05-31,0.165380,-0.165380,-0.496140,0.496140,-0.826901,0.826901,0.000000
...,...,...,...,...,...,...,...
2025-06-30,0.891450,-0.297150,0.594300,-0.891450,0.000000,0.297150,-0.594300
2025-07-31,0.000000,-0.569268,0.569268,-0.853902,0.853902,0.284634,-0.284634
2025-08-31,0.000000,-0.279018,0.558036,-0.837055,0.837055,0.279018,-0.558036


# Backtest

In [254]:
benchmark_TR = benchmark_TR.squeeze()
benchmark_XR = benchmark_XR.squeeze()

In [255]:
results = pmp.run_cc_strategy(
    weights = final_strategy_weights,
    returns = currency_XR,
    rf = riskfree,
    frequency = frequency,
    t_cost = t_cost, 
    benchmark = benchmark_XR,
    long_short = short,
    beta_neutral = beta_neutral
)

display(results)

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_US,w_UK,w_EU,w_JP,w_CH,w_AU,w_EM
Date,,,,,,,,,,,,,
1998-02-28,-0.004644,-0.004644,0.064065,0.000000,0.0,0.0039,0.111694,-0.111381,-0.554804,-0.333815,0.335190,0.553116,0.000000
1998-03-31,-0.024354,-0.024354,0.039375,0.640790,0.0,0.0039,0.339811,-0.339831,-0.557606,-0.102564,0.110804,0.549385,0.000000
1998-04-30,-0.014401,-0.014401,0.005630,0.728697,0.0,0.0043,0.337089,0.113707,-0.556778,-0.332238,-0.110984,0.549204,0.000000
1998-05-31,-0.041604,-0.041604,-0.015149,0.909782,0.0,0.0040,0.342808,-0.105326,-0.558537,0.112171,-0.336137,0.545022,0.000000
1998-06-30,0.013536,0.013536,0.020679,0.728246,0.0,0.0041,0.110496,-0.116254,-0.337601,0.333163,-0.546145,0.556341,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,0.047196,0.047196,0.040088,0.987283,0.0,0.0034,0.159668,-0.171646,0.505773,-0.494687,0.334559,0.000000,-0.333667
2025-07-31,0.032702,0.032702,0.009721,1.117460,0.0,0.0034,0.509396,-0.167421,0.327049,-0.491049,0.000000,0.163555,-0.341530
2025-08-31,-0.007337,-0.007337,0.022608,1.274096,0.0,0.0038,0.000000,-0.334121,0.335351,-0.501198,0.497330,0.167320,-0.164681


# Portfolio Statistics

In [256]:
pmp.run_perf_summary_benchmark_vs_strategy(results, alreadyXs = True)

,Benchmark,Strategy
Arithm Avg Total Return,6.9887,-0.0145
Arithm Avg Xs Return,4.9498,-2.0534
Std Xs Returns,15.5470,10.8307
Sharpe Arithmetic,0.3184,-0.1896
Geom Avg Total Return,5.9249,-0.6141
Geom Avg Xs Return,3.8686,-2.6704
Sharpe Geometric,0.2488,-0.2466
Min Xs Return,-19.0940,-18.8406
Max Xs Return,12.8084,8.8723
Skewness,-0.6049,-1.1302
